# Transfer Learning Example on Radar Dataset (Great Smoky Mountains)

## Importing the required modules

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import scipy.io as sio
import pandas as pd
import os
import re
import math
import itertools
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from itertools import product
from sklearn.utils.class_weight import compute_class_weight
torch.cuda.empty_cache()

# Reading the labels and creating train and test data

In [2]:
def index(string):
    s = re.findall("[0-9]", string)
    return int(''.join(s))

scenario_idx = 29 # Bonneville Salt Flats
names = os.listdir(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names = sorted(names, key=index)
print(len(names))
y = pd.read_csv(f'num{scenario_idx}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names = y.columns[4:7]
y = y[col_names].to_numpy()

scenario_idx_test = 76 # Great Smoky Mountains
names_test = os.listdir(f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names_test = sorted(names_test, key=index)
y_test = pd.read_csv(f'num{scenario_idx_test}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names_test = y_test.columns[4:7]
y_test = y_test[col_names_test].to_numpy()

# Define fine-tuning name and labels
few_shot_count = 64
names_FS = names_test[:few_shot_count]
y_FS = y_test[:few_shot_count]

# Create training and testing datasets
y_train = y[:int(0.9 * len(names))]
y_test = y_test[int(0.9 * len(names_test)) - 1:]
training_names = names[:int(0.9 * len(names))]
test_names = names_test[int(0.9 * len(names_test)) - 1:]

print('Training labels:')
print(y_train)

# Tensor Corners
##################################################################################
# num29: [10851, 215, -5.4], num60: [11073, 215, -5.3], num62: [11471, 215, -5.6]
# num76: [11388, 215, -6.15], num35: [11381, 215, -0.95]
##################################################################################

# Training dataset global constants
coord_tr = [10851, 215, -5.4]   # Tensor corner
rng_res_tr = 59.9585 / 2        # Range resolution
az_step_tr = 0.4                # Azimuth step size
el_step_tr = 0.01               # Elevation step size

# Test dataset global constants
coord_ts = [11388, 215, -6.15]  # Tensor corner
rng_res_ts = 59.9585 / 2        # Range resolution
az_step_ts = 0.4                # Azimuth step size
el_step_ts = 0.01               # Elevation step size

def Drawing_Batch(names, label, bs, ind, directory, normalize=True):
    x = []
    labels = []
    
    for j in range(ind * bs, (ind + 1) * bs):
        try:
            temp = sio.loadmat(directory + names[j])['P']
        except:
            break
        if normalize:
            Anorm = temp - np.min(temp.flatten())
            temp = np.divide(Anorm, np.max(Anorm.flatten()))
        x.append(temp)
        labels.append(label[j, :])
        
    x = torch.FloatTensor(np.array(x))
    labels = torch.FloatTensor(np.array(labels))
    return x, labels

100000
Training labels:
[[ 8.3355 13.325  19.525 ]
 [18.156  17.388  32.775 ]
 [ 8.777  24.73   18.582 ]
 ...
 [15.785  15.992  29.641 ]
 [ 3.1736 10.716  12.27  ]
 [ 4.5552 21.035  14.234 ]]


In [3]:
print(len(y_train))
print(len(y_test))

90000
10001


## Define a regression CNN and instantiating it

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(21, 32, 3, 1)
        self.conv2 = nn.Conv1d(32, 64, 3, 1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64 * 5, 20)  # Adjust input size based on the output of conv layers and max pooling
        self.fc2_reg = nn.Linear(20, 2)  # Adjusted output size

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = F.max_pool1d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool1d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output_reg = self.fc2_reg(x)  # (bs, 2)
        
        return output_reg
    
from torchsummary import summary
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = Net()
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
print(summary(model,(21,26)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 24]           2,048
       BatchNorm1d-2               [-1, 32, 24]              64
            Conv1d-3               [-1, 64, 10]           6,208
       BatchNorm1d-4               [-1, 64, 10]             128
            Linear-5                   [-1, 20]           6,420
            Linear-6                    [-1, 2]              42
               Net-7                    [-1, 2]               0
Total params: 14,910
Trainable params: 14,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.06
Estimated Total Size (MB): 0.08
----------------------------------------------------------------
None


## Global Definitions

In [14]:
bs = 128  # batch_size
num_epoch = 10  # number of epochs
PATH = './ckpt_model.pth'   # forsaving the model
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Define a Loss function and optimizer; Using GPU or CPU
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    
def Spher2Cart_1D(spherical):
    cartesian = np.zeros(3)
    hypotenuse = np.cos(np.radians(spherical[2]))*spherical[0]
    cartesian[0] = np.cos(np.radians(spherical[1]))*hypotenuse
    cartesian[1] = -np.sin(np.radians(spherical[1]))*hypotenuse
    cartesian[2] = np.sin(np.radians(spherical[2]))*spherical[0]
    return cartesian

## Train and evaluate the network

In [15]:
def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

start = time.time()
main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True)
end = time.time()
print(end - start)

Epoch 0/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8201.78965758  7752.09649026 -1007.12000372]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8673.90116691  7484.90196981 -1131.15709508]
Train Loss: 7.494222 ---- Test Loss: 41.460526
Epoch 1/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8190.91999719  7758.68701287 -1006.81964003]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8720.74150625  7430.90880575 -1131.19767986]
Train Loss: 0.218877 ---- Test Loss: 54.987772
Epoch 2/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8195.53088076  7748.54268886 -1006.4962598 ]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8777.59706943  7377.55012236 -1132.07939997]
Train Loss: 0.148049 ---- Test Loss: 60.940435
Epoch 3/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8199.69832416  7736.50556766 -1006.02905223]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8804.46039359  7362.15771869 -1133.13557497]
Train Loss: 0.12527

In [7]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 7.236562654693393
Localization Error (m) = 601.5837993848022


## Fine-Tuning Regression CNN

In [8]:
model.module.conv1.weight.requires_grad = False
model.module.conv1.bias.requires_grad = False
model.module.batchnorm1.weight.requires_grad = False
model.module.batchnorm1.bias.requires_grad = False

model.module.conv2.weight.requires_grad = False
model.module.conv2.bias.requires_grad = False
model.module.batchnorm2.weight.requires_grad = False
model.module.batchnorm2.bias.requires_grad = False

In [9]:
bs = 64  # batch_size
num_epoch = 100
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

main(names_FS, test_names, bs, num_epoch, y_FS, y_test, normalize=True)

Epoch 0/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8086.80003282  7779.37301071  -960.35345043]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8733.5095606   7429.46069382 -1126.2616375 ]
Train Loss: 23.463051 ---- Test Loss: 45.568877
Epoch 1/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8052.6115664   7808.18439468  -959.9617904 ]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8719.36172319  7454.31316165 -1126.7882673 ]
Train Loss: 21.562136 ---- Test Loss: 44.080161
Epoch 2/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8019.21554544  7835.30774769  -959.53276358]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8697.7510198   7484.21687488 -1127.08929463]
Train Loss: 19.887175 ---- Test Loss: 41.385051
Epoch 3/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-7987.56295577  7859.75791712  -959.06353462]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8669.49963237  7521.94851784 -1127.4126501 ]
Train Loss: 

[-8271.55879786  8100.10882815 -1137.17137312]
[-8306.40868441  7993.3010468  -1132.31997627]
Train Loss: 8.707473 ---- Test Loss: 12.213678
Epoch 33/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8015.64940276  7750.29825865  -954.24301021]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8306.69258534  7996.52749019 -1132.5598395 ]
Train Loss: 8.685439 ---- Test Loss: 12.108832
Epoch 34/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8019.93066026  7750.05326084  -954.4918856 ]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8306.60050758  7999.63614927 -1132.76511495]
Train Loss: 8.667862 ---- Test Loss: 12.016105
Epoch 35/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8023.3212501   7750.27651197  -954.71385067]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8306.18230727  8002.54203025 -1132.93354716]
Train Loss: 8.652351 ---- Test Loss: 11.934167
Epoch 36/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8025.8935

[-8271.55879786  8100.10882815 -1137.17137312]
[-8294.47743985  8037.7589841  -1134.51041809]
Train Loss: 8.267944 ---- Test Loss: 11.366908
Epoch 65/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8029.37483846  7797.66814824  -957.90782668]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8295.34802919  8037.67803555 -1134.56629712]
Train Loss: 8.263832 ---- Test Loss: 11.373819
Epoch 66/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8030.44568092  7797.87700716  -957.98602703]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8296.07142215  8037.57517399 -1134.61029799]
Train Loss: 8.260107 ---- Test Loss: 11.380546
Epoch 67/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8031.36365971  7797.99839187  -958.04962882]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8296.62674571  8037.49018213 -1134.64366582]
Train Loss: 8.256663 ---- Test Loss: 11.386544
Epoch 68/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8032.1062

[-8271.55879786  8100.10882815 -1137.17137312]
[-8297.22072685  8042.21777773 -1135.00871779]
Train Loss: 8.201628 ---- Test Loss: 11.330634
Epoch 97/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8039.65245673  7791.83861314  -958.19174302]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8296.98672258  8042.58731545 -1135.01747688]
Train Loss: 8.200289 ---- Test Loss: 11.326598
Epoch 98/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8039.51361457  7791.8205192   -958.18213254]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8296.70983167  8042.97936333 -1135.0247519 ]
Train Loss: 8.198958 ---- Test Loss: 11.322558
Epoch 99/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8039.31155481  7791.84620252  -958.17124456]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8296.4046286   8043.38104683 -1135.03069098]
Train Loss: 8.197637 ---- Test Loss: 11.318667


In [10]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 1.6143039394028127
Localization Error (m) = 169.2359471604409
